In [1]:
from pyAudioAnalysis import audioFeatureExtraction as aFE, audioBasicIO, \
audioAnalysis,audioSegmentation,audioVisualization,audioTrainTest
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import librosa
import numpy as np
import keras
from keras import models,layers
import os
from keras.utils import to_categorical
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import LSTM, Dense, Flatten, Dropout, GRU
from sklearn.metrics import confusion_matrix
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.ensemble import ExtraTreesClassifier

/Users/luigibungaro/dsr/lib/python3.6/site-packages/pydub/utils.py:165: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)
Using TensorFlow backend.


# Amplitude

In [5]:
def amplitude_data_all():
    target=[]
    i=0
    data=[]

    for item in os.listdir('peterbertley_Audio/Atraining_murmur'):  #Atraining_normal   #Atraining_extrahls
         if item.endswith(".wav"):
                Fs, X = audioBasicIO.readAudioFile('peterbertley_Audio/Atraining_murmur/'+str(item))
                data.append(X)
                target.append(['0','murmur'])

    for item in os.listdir('peterbertley_Audio/Atraining_normal'):    #Atraining_extrahls
         if item.endswith(".wav"):
                Fs, X = audioBasicIO.readAudioFile('peterbertley_Audio/Atraining_normal/'+str(item))
                data.append(X)
                target.append(['1','normal'])

    for item in os.listdir('peterbertley_Audio/Atraining_extrahls'):   
         if item.endswith(".wav"):
                Fs, X = audioBasicIO.readAudioFile('peterbertley_Audio/Atraining_extrahls/'+str(item))
                data.append(X)
                target.append(['2','extrahls'])
          
    #Preprocessing
    threshold = 5*Fs   #5 seconds threshols
    sampling_indexes = np.arange(0,threshold,10)
    drop=[]
    for k,i in enumerate(data):
        if len(i)<threshold:
            drop.append(k)
    drop = drop[::-1]  #reversing the order for the dropping
    print('shape before dropping:',len(data),len(target))
    for i in drop:
        data.pop(i)
        target.pop(i)
    print('shape after dropping:',len(data),len(target))

    # Making all the arrays of the same lengths, and sampling every 50 Hertz

    for k,i in enumerate(data):
        data[k]=i[:threshold]
    data = np.array(data)

    # StandardScaler
    scaler = StandardScaler()
    data = scaler.fit_transform(data)

    print('final shape of your data is: ',data.shape)

    #Setting the target with categorical transformation

    target = np.array(target)
    target = target[:,0]

    target = to_categorical(target)
    print('final categorical target shape is:',target.shape)

    # Splitting data in training and test set

    X_train,X_test,y_train,y_test = train_test_split(data,target,test_size=0.4)
    return X_train,X_test,y_train,y_test

In [3]:
def amplitude_data_sample_mean():
    target=[]
    i=0
    data=[]

    for item in os.listdir('peterbertley_Audio/Atraining_murmur'):  #Atraining_normal   #Atraining_extrahls
         if item.endswith(".wav"):
                Fs, X = audioBasicIO.readAudioFile('peterbertley_Audio/Atraining_murmur/'+str(item))
                data.append(X)
                target.append(['0','murmur'])

    for item in os.listdir('peterbertley_Audio/Atraining_normal'):    #Atraining_extrahls
         if item.endswith(".wav"):
                Fs, X = audioBasicIO.readAudioFile('peterbertley_Audio/Atraining_normal/'+str(item))
                data.append(X)
                target.append(['1','normal'])

    for item in os.listdir('peterbertley_Audio/Atraining_extrahls'):   
         if item.endswith(".wav"):
                Fs, X = audioBasicIO.readAudioFile('peterbertley_Audio/Atraining_extrahls/'+str(item))
                data.append(X)
                target.append(['2','extrahls'])
    #Preprocessing
    threshold = 5*Fs   #5 seconds threshols
    sampling_indexes = np.arange(0,threshold,10)
    drop=[]
    for k,i in enumerate(data):
        if len(i)<threshold:
            drop.append(k)
    drop = drop[::-1]  #reversing the order for the dropping
    print('shape before dropping:',len(data),len(target))
    for i in drop:
        data.pop(i)
        target.pop(i)
    print('shape after dropping:',len(data),len(target))

    # Making all the arrays of the same lengths, and sampling every 50 Hertz

    for k,i in enumerate(data):
        data[k]=i[:threshold]
        data[k]=i[sampling_indexes]
    data = np.array(data)

    # StandardScaler
    scaler = StandardScaler()
    data = scaler.fit_transform(data)

    print('final shape of your data is: ',data.shape)

    #Setting the target with categorical transformation

    target = np.array(target)
    target = target[:,0]

    target = to_categorical(target)
    print('final categorical target shape is:',target.shape)

    # Splitting data in training and test set

    X_train,X_test,y_train,y_test = train_test_split(data,target,test_size=0.4)
    return X_train,X_test,y_train,y_test

In [4]:
def amplitude_data_sample_asis():
    target=[]
    i=0
    data=[]

    for item in os.listdir('peterbertley_Audio/Atraining_murmur'):  #Atraining_normal   #Atraining_extrahls
         if item.endswith(".wav"):
                Fs, X = audioBasicIO.readAudioFile('peterbertley_Audio/Atraining_murmur/'+str(item))
                X_temp = X[1:]
                data.append(X_temp-X[:-1])
                target.append(['0','murmur'])

    for item in os.listdir('peterbertley_Audio/Atraining_normal'):    #Atraining_extrahls
         if item.endswith(".wav"):
                Fs, X = audioBasicIO.readAudioFile('peterbertley_Audio/Atraining_normal/'+str(item))
                #X_temp = X[1:]
                #data.append(X_temp-X[:-1])
                data.append(data)
                target.append(['1','normal'])

    for item in os.listdir('peterbertley_Audio/Atraining_extrahls'):   
         if item.endswith(".wav"):
                Fs, X = audioBasicIO.readAudioFile('peterbertley_Audio/Atraining_extrahls/'+str(item))
                X_temp = X[1:]
                data.append(X_temp-X[:-1])
                target.append(['2','extrahls'])
    #Preprocessing
    threshold = 5*Fs   #5 seconds threshols
    sampling_indexes = np.arange(0,threshold,10)
    drop=[]
    for k,i in enumerate(data):
        if len(i)<threshold:
            drop.append(k)
    drop = drop[::-1]  #reversing the order for the dropping
    print('shape before dropping:',len(data),len(target))
    for i in drop:
        data.pop(i)
        target.pop(i)
    print('shape after dropping:',len(data),len(target))

    # Making all the arrays of the same lengths, and sampling every 50 Hertz

    for k,i in enumerate(data):
        data[k]=i[:threshold]
        #data[k]=i[sampling_indexes]
    data = np.array(data)

    # StandardScaler
    scaler = StandardScaler()
    data = scaler.fit_transform(data)

    print('final shape of your data is: ',data.shape)

    #Setting the target with categorical transformation

    target = np.array(target)
    target = target[:,0]

    target = to_categorical(target)
    print('final categorical target shape is:',target.shape)

    # Splitting data in training and test set

    X_train,X_test,y_train,y_test = train_test_split(data,target,test_size=0.4)
    return X_train,X_test,y_train,y_test

In [ ]:
def amplitude_data_difference():
    target=[]
    i=0
    data=[]

    for item in os.listdir('peterbertley_Audio/Atraining_murmur'):  #Atraining_normal   #Atraining_extrahls
         if item.endswith(".wav"):
                Fs, X = audioBasicIO.readAudioFile('peterbertley_Audio/Atraining_murmur/'+str(item))
                X_temp = X[1:]
                data.append(X_temp-X[:-1])
                target.append(['0','murmur'])

    for item in os.listdir('peterbertley_Audio/Atraining_normal'):    #Atraining_extrahls
         if item.endswith(".wav"):
                Fs, X = audioBasicIO.readAudioFile('peterbertley_Audio/Atraining_normal/'+str(item))
                #X_temp = X[1:]
                #data.append(X_temp-X[:-1])
                data.append(data)
                target.append(['1','normal'])

    for item in os.listdir('peterbertley_Audio/Atraining_extrahls'):   
         if item.endswith(".wav"):
                Fs, X = audioBasicIO.readAudioFile('peterbertley_Audio/Atraining_extrahls/'+str(item))
                X_temp = X[1:]
                data.append(X_temp-X[:-1])
                target.append(['2','extrahls'])
    #Preprocessing
    threshold = 5*Fs   #5 seconds threshols
    sampling_indexes = np.arange(0,threshold,10)
    drop=[]
    for k,i in enumerate(data):
        if len(i)<threshold:
            drop.append(k)
    drop = drop[::-1]  #reversing the order for the dropping
    print('shape before dropping:',len(data),len(target))
    for i in drop:
        data.pop(i)
        target.pop(i)
    print('shape after dropping:',len(data),len(target))

    # Making all the arrays of the same lengths, and sampling every 50 Hertz

    for k,i in enumerate(data):
        data[k]=i[:threshold]
        #data[k]=i[sampling_indexes]
    data = np.array(data)

    # StandardScaler
    scaler = StandardScaler()
    data = scaler.fit_transform(data)

    print('final shape of your data is: ',data.shape)

    #Setting the target with categorical transformation

    target = np.array(target)
    target = target[:,0]

    target = to_categorical(target)
    print('final categorical target shape is:',target.shape)

    # Splitting data in training and test set

    X_train,X_test,y_train,y_test = train_test_split(data,target,test_size=0.4)
    return X_train,X_test,y_train,y_test